In [1]:
import pandas as pd
import numpy as np

url = '.\대출금리\중소기업 대출\기업대출금리(일반신용-중소기업).csv'
class read():
    credit_distribution = pd.read_csv('.\기업신용등급분포.csv', encoding='cp949')
    cd = np.array(credit_distribution['구성 비율']).astype('float')
    

    def __init__(self,url):
        self.loan_dev = url

    def tidy(self):
        self.loan_dev = pd.read_csv(self.loan_dev,encoding='cp949')
        self.loan_dev['은행'] = self.loan_dev['은행'].str.strip()
        self.loan_dev.columns = ['은행','구분','1~3등급','4등급','5등급','6등급','7~10등급','평균금리','참고사항']
        self.loan_dev.drop([0],axis=0,inplace=True)
        self.loan_dev.drop(['참고사항','구분'],axis=1,inplace=True)
        self.loan_dev['평균금리']=self.loan_dev['평균금리'].replace('-',np.NaN)
        self.loan_dev.dropna(axis=0,inplace=True)
        self.loan_dev.reset_index(drop=True,inplace=True)
        self.loan_dev.replace('-',0,inplace=True)
        self.interest_grade = self.loan_dev.loc[:,'1~3등급':'7~10등급']
        return self.loan_dev

    def multi(self):
        self.a = []
        for i in range(len(self.interest_grade)):
            self.rate_grade = np.array(self.interest_grade.loc[i:i,'1~3등급':'7~10등급'], dtype='float')[0]
            #은행 상품 단위 신용 등급 별 이자율
            self.boolean = self.rate_grade.astype('bool')
            #상품의 대상 고객 
            self.ac_cd = sum(self.boolean * self.cd)
            # 전체 신용 등급 대비 타겟 고객 비율
            self.customer_ration = self.cd / self.ac_cd
            # 타겟 고객 대비 특정 신용 등급 고객 비율
            self.real_income = self.rate_grade @ self.customer_ration
            # 대출 금리에 대한 실제 고객의 가중 평균으로 구한 상품 수익률
            self.a.append(self.real_income)
        
        self.loan_dev['가중평균수익'] = self.a
        return self.loan_dev


a = read(url)
a.tidy()
a.multi()

,은행,1~3등급,4등급,5등급,6등급,7~10등급,평균금리,가중평균수익
0,BNK경남은행,3.22,3.56,4.06,5.22,5.80,3.74,4.718084
1,BNK부산은행,3.62,4.51,4.75,5.39,7.64,4.36,5.112930
2,DGB대구은행,3.90,3.99,4.68,5.85,9.06,4.69,5.357745
3,IBK기업은행,3.55,4.43,5.94,8.77,6.80,5.07,7.494988
4,KB국민은행,2.73,3.99,5.56,8.90,7.67,4.52,7.414171
5,KDB산업은행,2.86,3.41,4.04,6.11,9.91,3.54,5.257896
6,NH농협은행,3.32,3.77,5.04,7.21,10.03,4.89,6.264918
7,SH수협은행,3.79,4.89,5.20,6.79,7.71,4.79,6.114519
8,광주은행,3.97,4.05,5.03,5.55,6.34,4.61,5.274511
9,스탠다드차타드은행,0,0,0,0,4.60,4.93,4.600000
